# ЛР 6

ухудшаем качество фоток:

In [2]:
from PIL import Image
import os

directory = 'new_images'  # Папка с изображениями

for filename in os.listdir(directory):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Фильтрация файлов по форматам изображений
        filepath = os.path.join(directory, filename)
        img = Image.open(filepath)
        width_percent = 400 / float(img.size[0])  # Рассчитываем процентное соотношение изменения размера
        height_size = int(float(img.size[1]) * float(width_percent))  # Рассчитываем высоту пропорционально ширине 150 пикселей
        img = img.resize((400, height_size), Image.LANCZOS)  # Изменяем размер изображения
        img.save('worse_images/' + filename)  # Сохраняем измененное изображение обратно в файл


In [3]:
import cv2
import os
from tqdm import tqdm

# Функция для извлечения SIFT-дескрипторов и сохранения их в файле .sift
def extract_sift_descriptors(image_path):
    # Чтение изображения
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Инициализация детектора и описателя SIFT
    sift = cv2.SIFT_create()

    # Обнаружение ключевых точек и вычисление дескрипторов
    keypoints, descriptors = sift.detectAndCompute(image, None)

    # Получение имени файла без расширения
    filename = os.path.splitext(image_path)[0]

    # Сохранение дескрипторов в файле .sift
    sift_filename = f"{filename}.sift"
    with open(sift_filename, 'w') as file:
        for descriptor in descriptors:
            descriptor_line = ' '.join([str(value) for value in descriptor])
            file.write(f"{descriptor_line}\n")

# Путь к папке с изображениями
folder_path = 'worse_images/'

# Получение списка файлов изображений в папке
image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.jpg')]

# Извлечение SIFT-дескрипторов для каждого изображения
for image_file in tqdm(image_files):
    extract_sift_descriptors(image_file)


100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00,  7.47it/s]


In [4]:
import os

# Путь к папке с изображениями
folder_path = 'worse_images/'

# Получение списка файлов изображений в папке
image_files = [file for file in os.listdir(folder_path) if file.endswith('.jpg')]

# Создание списка имен файлов изображений
imlist = [os.path.join(folder_path, file) for file in image_files]


In [5]:
imlist

['worse_images/image_1.jpg',
 'worse_images/image_2.jpg',
 'worse_images/image_3.jpg',
 'worse_images/image_4.jpg',
 'worse_images/image_5.jpg',
 'worse_images/image_6.jpg',
 'worse_images/image_7.jpg']

In [6]:
import cv2
import numpy as np
import os

# Путь к папке с файлами .sift
sift_folder_path = 'worse_images/'

# Получение списка файлов .sift
sift_files = [file for file in os.listdir(sift_folder_path) if file.endswith('.sift')]

# Чтение дескрипторов и сопоставление
nbr_images = len(sift_files)
matchscores = np.zeros((nbr_images, nbr_images))

for i in range(nbr_images):
    for j in range(i + 1, nbr_images):
        sift_filename1 = os.path.join(sift_folder_path, sift_files[i])
        sift_filename2 = os.path.join(sift_folder_path, sift_files[j])

        # Чтение дескрипторов из файлов .sift
        with open(sift_filename1, 'r') as file:
            d1 = np.array([[float(num) for num in line.split()] for line in file.readlines()])
        with open(sift_filename2, 'r') as file:
            d2 = np.array([[float(num) for num in line.split()] for line in file.readlines()])

        print(sift_filename1, sift_filename2)
        d1 = d1.astype(np.float32)
        d2 = d2.astype(np.float32)
        # Сопоставление дескрипторов
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(d1, d2, k=2)
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
        nbr_matches = len(good_matches)
        print('Число соответствий =', nbr_matches, '\n')
        matchscores[i, j] = nbr_matches

# Копирование значений для нижнего треугольника
for i in range(nbr_images):
    for j in range(i + 1, nbr_images):
        matchscores[j, i] = matchscores[i, j]
    matchscores[i, i] = np.inf

worse_images/image_1.sift worse_images/image_2.sift
Число соответствий = 3 

worse_images/image_1.sift worse_images/image_3.sift
Число соответствий = 9 

worse_images/image_1.sift worse_images/image_4.sift
Число соответствий = 4 

worse_images/image_1.sift worse_images/image_5.sift
Число соответствий = 5 

worse_images/image_1.sift worse_images/image_6.sift
Число соответствий = 5 

worse_images/image_1.sift worse_images/image_7.sift
Число соответствий = 3 

worse_images/image_2.sift worse_images/image_3.sift
Число соответствий = 12 

worse_images/image_2.sift worse_images/image_4.sift
Число соответствий = 8 

worse_images/image_2.sift worse_images/image_5.sift
Число соответствий = 6 

worse_images/image_2.sift worse_images/image_6.sift
Число соответствий = 8 

worse_images/image_2.sift worse_images/image_7.sift
Число соответствий = 14 

worse_images/image_3.sift worse_images/image_4.sift
Число соответствий = 3 

worse_images/image_3.sift worse_images/image_5.sift
Число соответствий = 4

In [7]:
matchscores

array([[inf,  3.,  9.,  4.,  5.,  5.,  3.],
       [ 3., inf, 12.,  8.,  6.,  8., 14.],
       [ 9., 12., inf,  3.,  4.,  2.,  4.],
       [ 4.,  8.,  3., inf, 20.,  9., 41.],
       [ 5.,  6.,  4., 20., inf,  7., 10.],
       [ 5.,  8.,  2.,  9.,  7., inf, 14.],
       [ 3., 14.,  4., 41., 10., 14., inf]])

In [8]:
!pip install pydot

In [12]:
!pip install graphviz

In [13]:
import pydot
from PIL import Image
import os

threshold = 12
g = pydot.Dot(graph_type='graph')

# Добавление изображений к узлам
for i in range(nbr_images):
    for j in range(i + 1, nbr_images):
        if matchscores[i, j] > threshold:
            # Получение имен файлов
            filename1 = os.path.basename(imlist[i])  # Имя файла первого изображения
            filename2 = os.path.basename(imlist[j])  # Имя файла второго изображения

            # Создание узлов с текстовыми значениями
            g.add_node(pydot.Node(str(i), fontcolor='black', shape='rectangle', label=filename1))
            g.add_node(pydot.Node(str(j), fontcolor='black', shape='rectangle', label=filename2))

            # Добавление ребра между узлами
            g.add_edge(pydot.Edge(str(i), str(j)))

# Сохранение графа как PNG
g.write_png('result.png')


FileNotFoundError: [WinError 2] "dot" not found in path.

---

In [11]:
import pydot
from PIL import Image  # Import Image from PIL module

threshold = 120  # мин. число соответствий для создания ребра
g = pydot.Dot(graph_type='graph')  # не нужно строить ориентированный граф, как предполагается по умолчанию

for i in range(nbr_images):
    for j in range(i + 1, nbr_images):
        if matchscores[i, j] > threshold:
            # первое изображение в паре
            im = Image.open(imlist[i])
            im.thumbnail((100, 100))
            filename = str(i) + '.png'
            #im.save(filename)  # нужны временные файлы подходящего размера
            g.add_node(pydot.Node(str(i), fontcolor='transparent', shape='rectangle', image=filename))

            # второе изображение в паре
            im = Image.open(imlist[j])
            im.thumbnail((100, 100))
            filename = str(j) + '.png'
            #im.save(filename)  # нужны временные файлы подходящего размера
            g.add_node(pydot.Node(str(j), fontcolor='transparent', shape='rectangle', image=filename))

            g.add_edge(pydot.Edge(str(i), str(j)))

# Save the graph as PNG
g.write_png('result_img.png')


FileNotFoundError: [WinError 2] "dot" not found in path.